In [ ]:
!pip install gpustat
!gpustat

In [ ]:
import os
import numpy as np
from tensorflow import keras
import imgaug as ia
from matplotlib import pyplot as plt

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
INPUT_SIZE = (384, 512)
CLASS_COUNT = 6
DATA_DIR = '/content/gdrive/My Drive/data_source/garbage_classifier/training_data/'
SAMPLE_PATH = '/content/gdrive/My Drive/data_source/garbage_classifier/samples.npy'
BATCH_SIZE = 4
VALIDATION_SPLIT = 0.1

LOG_PATH = '/content/gdrive/My Drive/data_source/garbage_classifier/resnet_34_log/'
MODEL_ARCHIVE_PATH = '/content/gdrive/My Drive/data_source/garbage_classifier/resnet_34_models/'

In [ ]:
def image_preprocessing(images):
    sometimes = lambda x: ia.augmenters.Sometimes(0.5, x)
    augmentation_sequence = ia.augmenters.Sequential([
        sometimes(ia.augmenters.AdditiveGaussianNoise(loc=(0.0, 0.1))),
        sometimes(ia.augmenters.Add(-0.1, 0.1))
    ])
    return augmentation_sequence(images=images)

data_generator = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=image_preprocessing,
    validation_split=VALIDATION_SPLIT
)
data_generator.fit(np.load(SAMPLE_PATH))
flow_args = {
    'directory': DATA_DIR,
    'target_size': INPUT_SIZE,
    'color_mode': 'rgb',
    'batch_size': BATCH_SIZE,
    'class_mode': 'categorical',
    'save_format': 'jpeg'
}

training_data_generator = data_generator.flow_from_directory(
    **flow_args,
    subset='training'
)
validation_data_generator = data_generator.flow_from_directory(
    **flow_args,
    subset='validation'
)
print(training_data_generator.class_indices)

In [ ]:
def _conv_batchnorm(filters, kernel_size, strides=(1, 1), padding='valid', kernel_regularizer=None, activation='linear'):
    def structure(input_tensor):
        layer = keras.layers.Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            kernel_initializer='he_normal',
            kernel_regularizer=kernel_regularizer
        )(input_tensor)
        layer = keras.layers.BatchNormalization()(layer)
        layer = keras.layers.Activation(activation)(layer)
        return layer
    return structure

def _residual_block(filters, kernel_size, kernel_regularizer=None, is_beginning_block=False):
    assert len(filters) == len(kernel_size)
    def structure(input_tensor):
        residual, layer = input_tensor, input_tensor
        for index, (f, k) in enumerate(zip(filters, kernel_size)):
            layer = _conv_batchnorm(
                filters=f,
                kernel_size=k,
                strides=(2, 2) if is_beginning_block and index == 0 else (1, 1),
                padding='same',
                kernel_regularizer=kernel_regularizer,
                activation='linear' if index == len(filters) - 1 else 'relu'
            )(layer)
        if tuple(residual.shape) != tuple(layer.shape):
            residual = _conv_batchnorm(
                filters=layer.shape[-1],
                kernel_size=(1, 1),
                strides=[*map(lambda x: int(round(x[0] / x[1])), [*zip(residual.shape, layer.shape)][1:3])],
                kernel_regularizer=kernel_regularizer
            )(residual)
        layer = keras.layers.Add()([residual, layer])
        layer = keras.layers.Activation('relu')(layer)
        return layer
    return structure

def _resnet(input_size, class_count, layer_filter_kernels):
    model_input = keras.layers.Input(shape=input_size)
    layer = _conv_batchnorm(
        filters=64,
        kernel_size=(7, 7),
        strides=(2, 2),
        padding='same',
        kernel_regularizer=keras.regularizers.l2(1e-4),
        activation='relu'
    )(model_input)
    layer = keras.layers.MaxPool2D(
        pool_size=(3, 3),
        strides=(2, 2),
        padding='same'
    )(layer)
    for block_index, (layer_count, filters, kernel_size) in enumerate(layer_filter_kernels):
        for layer_index in range(layer_count):
            layer = _residual_block(
                filters=filters,
                kernel_size=kernel_size,
                kernel_regularizer=keras.regularizers.l2(1e-4),
                is_beginning_block=(layer_index == 0 and block_index != 0)
            )(layer)
    layer = keras.layers.GlobalAveragePooling2D()(layer)
    layer = keras.layers.Dense(
        units=class_count,
        activation='softmax'
    )(layer)
    model = keras.models.Model(inputs=model_input, outputs=layer)
    return model

def resnet_34(input_size, class_count):
    return _resnet(
        input_size, 
        class_count, 
        layer_filter_kernels=[
            (3, [64], [(3, 3)]), 
            (4, [128], [(3, 3)]), 
            (6, [256], [(3, 3)]), 
            (3, [512], [(3, 3)])
        ]
    )

def resnet_50(input_size, class_count):
    return _resnet(
        input_size,
        class_count,
        layer_filter_kernels=[
            (3, [64, 64, 256], [(1, 1), (3, 3), (1, 1)]),
            (4, [128, 128, 512], [(1, 1), (3, 3), (1, 1)]),
            (6, [256, 256, 1024], [(1, 1), (3, 3), (1, 1)]),
            (3, [512, 512, 2048], [(1, 1), (3, 3), (1, 1)])
        ]
    )

def resnet_101(input_size, class_count):
    return _resnet(
        input_size,
        class_count,
        layer_filter_kernels=[
            (3, [64, 64, 256], [(1, 1), (3, 3), (1, 1)]),
            (4, [128, 128, 512], [(1, 1), (3, 3), (1, 1)]),
            (23, [256, 256, 1024], [(1, 1), (3, 3), (1, 1)]),
            (3, [512, 512, 2048], [(1, 1), (3, 3), (1, 1)])
        ]
    )

def resnet_152(input_size, class_count):
    return _resnet(
        input_size,
        class_count,
        layer_filter_kernels=[
            (3, [64, 64, 256], [(1, 1), (3, 3), (1, 1)]),
            (8, [128, 128, 512], [(1, 1), (3, 3), (1, 1)]),
            (36, [256, 256, 1024], [(1, 1), (3, 3), (1, 1)]),
            (3, [512, 512, 2048], [(1, 1), (3, 3), (1, 1)])
        ]
    )

In [ ]:
model = resnet_34((*INPUT_SIZE, 3), CLASS_COUNT)
model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.categorical_crossentropy, 
    metrics=['accuracy']
)

In [ ]:
model.fit(
    training_data_generator,
    validation_data=validation_data_generator,
    epochs=128,
    steps_per_epoch=512,
    validation_steps=64,
    callbacks=[
        keras.callbacks.LearningRateScheduler(lambda ei, lr: lr / 2.0 if ei > 0 and (ei / 32).is_integer() else lr),
        keras.callbacks.TensorBoard(log_dir=LOG_PATH, update_freq=64),
        keras.callbacks.ModelCheckpoint(os.path.join(MODEL_ARCHIVE_PATH, 'model_checkpoint_{epoch:02d}.hdf5'), period=16)
    ]
)